In [1]:
import pandas as pd

# Load CSV files into DataFrames
df_lang = pd.read_csv('/content/HCK_HEC_LANG.csv')
df_skills = pd.read_csv('/content/HCK_HEC_SKILLS.csv')
df_staffing = pd.read_csv('/content/HCK_HEC_STAFFING.csv')
df_user = pd.read_csv('/content/HCK_HEC_USER.csv')
df_xp = pd.read_csv('/content/HCK_HEC_XP.csv')

## Function to pivot tables

In [2]:
import pandas as pd

def pivot_skills_table(df_skills: pd.DataFrame) -> pd.DataFrame:
    """
    Pivot the skills table to ensure all skills are included, even if no current user has them.
    """
    # Step 1: Get all unique skills
    all_skills = df_skills["SKILLS_DSC"].unique()

    # Step 2: Create dummy rows with a fake user to ensure all skills appear
    dummy_user_id = -99999
    dummy_rows = pd.DataFrame({
        "USER_ID": [dummy_user_id] * len(all_skills),
        "SKILLS_DSC": all_skills,
        "LEVEL_VAL": [0] * len(all_skills)
    })

    # Step 3: Append dummy to original skills table
    df_skills_augmented = pd.concat([df_skills, dummy_rows], ignore_index=True)

    # Step 4: Pivot
    skills_pivot = df_skills_augmented.pivot_table(
        index="USER_ID",
        columns="SKILLS_DSC",
        values="LEVEL_VAL",
        aggfunc='max',
        fill_value=0
    )

    # Step 5: Drop the dummy user and rename columns
    skills_pivot = skills_pivot.drop(index=dummy_user_id)
    skills_pivot.columns = [f"{col}_level" for col in skills_pivot.columns]

    return skills_pivot.reset_index()

def pivot_language_table(df_lang: pd.DataFrame) -> pd.DataFrame:
    """
    Pivot the language table to ensure English and French columns are always included,
    even if no users currently have those language scores.
    """
    # Define the languages you want to ensure are always present
    required_languages = ["English", "French"]

    # Step 1: Create dummy rows for each language
    dummy_user_id = -99999
    dummy_rows = pd.DataFrame({
        "USER_ID": [dummy_user_id] * len(required_languages),
        "LANGUAGE_SKILL_DSC": required_languages,
        "LANGUAGE_SKILL_LVL": [0] * len(required_languages)
    })

    # Step 2: Append dummy to original lang table
    df_lang_augmented = pd.concat([df_lang, dummy_rows], ignore_index=True)

    # Step 3: Pivot
    lang_pivot = df_lang_augmented.pivot_table(
        index="USER_ID",
        columns="LANGUAGE_SKILL_DSC",
        values="LANGUAGE_SKILL_LVL",
        aggfunc='max',
        fill_value=0
    )

    # Step 4: Drop the dummy user and rename columns
    lang_pivot = lang_pivot.drop(index=dummy_user_id, errors='ignore')
    lang_pivot = lang_pivot.rename(columns={
        "English": "English_level",
        "French": "French_level"
    })

    return lang_pivot.reset_index()


def merge_user_skills_languages(
    df_user: pd.DataFrame,
    skills_pivot: pd.DataFrame,
    lang_pivot: pd.DataFrame
) -> pd.DataFrame:
    """
    Merges the user base table with pivoted skills and language tables.
    Fills missing values with 0.
    """
    merged_df = df_user.merge(skills_pivot, on="USER_ID", how="left")
    merged_df = merged_df.merge(lang_pivot, on="USER_ID", how="left")
    merged_df.fillna(0, inplace=True)
    return merged_df


skills_pivoted = pivot_skills_table(df_skills)
lang_pivoted = pivot_language_table(df_lang)
Users_Lang_Skills = merge_user_skills_languages(df_user, skills_pivoted, lang_pivoted)

In [3]:
# Save the Users_Lang_Skills DataFrame as a CSV file
lang_pivoted.to_csv("lang_pivoted.csv", index=False)
skills_pivoted.to_csv("skills_pivoted.csv", index=False)
Users_Lang_Skills.to_csv("Users_Lang_Skills.csv", index=False)